# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [26]:
import urllib
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
#print(df_train)

### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
Try to understand each step of the learning algorithm and comment each line.

In [28]:
X_train.shape

(80, 22)

In [29]:
def learn_reg_ERM(X,y,lbda):
    
    max_iter = 200
    #boundary e 
    e  = 0.001
    #step size alpha
    alpha = 1.

    # initialize the weigths vector randomly. it must have the same number of rows as 
    # there are features in the input matrix = 22
    w = np.random.randn(X.shape[1])
    
    
    for iteration in np.arange(max_iter):
        # multiply input vector by the current weight vector
        h = np.dot(X,w)
        # calculate loss between predicted and true labels and gradient
        l,lg = loss(h, y)
        print ('loss: {}'.format(np.mean(l)))
        
        # compute l2-reglarizer and the gradient
        # for the current weight vector w
        r,rg = reg(w, lbda)
        
        # gradient
        g = np.dot(X.T,lg) + rg 
        
        # update learning rate
        if (iteration > 0):
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old))
        
        #update weights
        w = w - alpha * g
        if (np.linalg.norm(alpha * g) < e):
            break
        g_old = g
        
    return w

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss at the points $h_i$. The partial derivative of the hinge loss $h_i$ with respect to the $i$-th position of the weight vector $\textbf{w}$ is $g_{i} = −y x_{i}$ if $l_{i} > 0$, else $g_{i} = 0$).

In [30]:
def loss(h, y):
    #element-wise max
    l = np.maximum(0,1 - y * h)
    # calculate lg if l>0 (to go to min)
    g = - y *(l > 0) 
    
    return l, g

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [31]:
def reg(w, lbda):
    r = (lbda/2) * w.dot(w.T)
    g = lbda * w
    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [32]:
def predict(w, X):
    preds = 2 * (np.dot(X,w) > 0) - 1
    return preds

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [33]:
def accuracy(y, y_hat):
    return 100.0*np.sum(y_hat == y) / len(y)

In [34]:
# y_train = [-1 if y_i == 0 else 1 for y_i in y_train]
# y_test = [-1 if y_i == 0 else 1 for y_i in y_test]

# convert labels so that they are -1 or 1: 
# 2*0-1 = -1
# 2*1-1 = 1
y_train = 2 * y_train - 1
y_test = 2 * y_test - 1

w = learn_reg_ERM(X_train, y_train, 10)
y_hat_train = predict(w, X_train)
y_hat_test = predict(w, X_test)
print("Train Accuracy: {}", format(accuracy(y_train, y_hat_train)))
print("Test Accuracy: {}", format(accuracy(y_test, y_hat_test)))

loss: 1.5576119362468852
loss: 18.68129028284217
loss: 1.039890599433886
loss: 2.071929440475506
loss: 1.0345199866570092
loss: 0.7352964581685507
loss: 0.9253035851764533
loss: 0.7235173496350471
loss: 0.7204697936311102
loss: 0.7276301076450034
loss: 0.7219143076381125
loss: 0.7259103348702128
loss: 0.7227166431662727
loss: 0.7194895477747958
loss: 0.7220647659837247
loss: 0.7196513273761939
loss: 0.7191263892841399
loss: 0.7201065691851497
loss: 0.7203390107986596
loss: 0.719513800436338
loss: 0.7196012487686833
loss: 0.7194341061088071
loss: 0.7212500000000001
loss: 0.7556927314039703
loss: 0.7209965643538148
loss: 0.7184853931540486
loss: 0.7206541005616433
loss: 0.718911373474971
loss: 0.7184793836252833
loss: 0.7181860794690212
loss: 0.7188685481473039
loss: 0.7184938555595503
loss: 0.7185061460384293
loss: 0.718284319529039
loss: 0.7185248239226378
loss: 0.7184030403869033
loss: 0.7184292522304228
loss: 0.7183904564268488
loss: 0.7184345988749803
loss: 0.7183877520987169
loss: 

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [35]:
##################
#INSERT CODE HERE#
##################